In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import io
import unicodedata
import string
import re
import random
import os
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import Dataset
from torch.optim import lr_scheduler
import itertools
import glob
plt.switch_backend('agg')
import matplotlib.ticker as ticker

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
batch_size = 32
words_to_load = 100000
SOS_token = 0
EOS_token = 1
PAD_token = 2
UNK_token = 3
LR_RATE = 0.001
MAX_LENGTH = 100
encoder_hidden_size = 128
decoder_hidden_size = 128
teacher_forcing_ratio = 0.5

__Preprocess Data__

In [3]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "<pad>", 3: "<unk>"}
        self.n_words = 4  # Count SOS, EOS, pad and unk

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [5]:
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def readLangs(dataset, lang1, lang2):
    chinese = os.getcwd()+'/iwslt-zh-en/{}.tok.{}'.format(dataset, lang1)
    english = os.getcwd()+'/iwslt-zh-en/{}.tok.{}'.format(dataset, lang2)

    chinese_lines = open(chinese, encoding='utf-8').read().strip().split('\n')
    english_lines = open(english, encoding='utf-8').read().strip().split('\n')
    length = len(chinese_lines)

    pairs = [[chinese_lines[i], normalizeString(english_lines[i])] for i in range(length)]
    pairs = filterPairs(pairs)
    
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    return input_lang, output_lang, pairs

In [7]:
train_input_lang, train_output_lang, train_pairs = readLangs('train', 'zh', 'en')
val_input_lang, val_output_lang, val_pairs = readLangs('dev', 'zh', 'en')
test_input_lang, test_output_lang, test_pairs = readLangs('test', 'zh', 'en')

__Data Loader__

In [8]:
class NMTDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, input_lang, output_lang, pairs):
        """
        @param data_list_1: list of sentence 1 tokens 
        @param data_list_2: list of sentence 2 tokens
        @param target_list: list of review targets 

        """
        self.input_lang = input_lang
        self.output_lang = output_lang
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        input_sentence = self.pairs[key][0]
        input_indexes = [self.input_lang.word2index[word] for word in input_sentence.split(' ')]
        input_indexes.append(EOS_token)
        input_length = len(input_indexes)

        output_sentence = self.pairs[key][1]
        output_indexes = [self.output_lang.word2index[word] for word in output_sentence.split(' ')]
        output_indexes.append(EOS_token)
        output_length = len(output_indexes)
        return [input_indexes, input_length, output_indexes, output_length]

    
def NMTDataset_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    input_ls = []
    output_ls = []
    input_length_ls = []
    output_length_ls = []
    
    for datum in batch:
        input_length_ls.append(datum[1])
        output_length_ls.append(datum[3])
    
    #find max length in each batch
    max_input = sorted(input_length_ls)[-1]
    max_output = sorted(output_length_ls)[-1]
    
    # padding
    for datum in batch:
        padded_vec_input = np.pad(np.array(datum[0]), 
                                  pad_width=((0,max_input-datum[1])), 
                                  mode="constant", constant_values=2).tolist()
        padded_vec_output = np.pad(np.array(datum[2]), 
                                   pad_width=((0,max_output-datum[3])), 
                                   mode="constant", constant_values=2).tolist()
        input_ls.append(padded_vec_input)
        output_ls.append(padded_vec_output)
    return [torch.tensor(torch.from_numpy(np.array(input_ls)), device=device), 
            torch.tensor(input_length_ls, device=device), 
            torch.tensor(torch.from_numpy(np.array(output_ls)), device=device), 
            torch.tensor(output_length_ls, device=device)]

In [9]:
# create pytorch dataloader
train_dataset = NMTDataset(train_input_lang, train_output_lang, train_pairs)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size,
                                           collate_fn=NMTDataset_collate_func,
                                           shuffle=True)

val_dataset = NMTDataset(val_input_lang, val_output_lang, val_pairs)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=batch_size,
                                         collate_fn=NMTDataset_collate_func,
                                         shuffle=True)

__Embedding__

In [10]:
def load_embedding(ft_path, words_to_load):
    fin = io.open(ft_path, 'r', encoding='utf-8', newline='\n', errors='ignore')

    n, d = map(int, fin.readline().split())
    vocab_size = words_to_load + 4
    embedding_dim = d

    embedding_mat = np.zeros((vocab_size, embedding_dim))
    token2id = {}
    id2token = {}
    all_tokens = ['SOS', 'EOS', '<unk>', '<pad>']

    for i, line in enumerate(fin):
        if i >= words_to_load:
            break
        s = line.rstrip().split(' ')
        embedding_mat[i+4, :] = np.asarray(s[1:])
        token2id[s[0]] = i+4
        id2token[i+4] = s[0]
        all_tokens.append(s[0])

        token2id['<pad>'] = PAD_token 
        token2id['<unk>'] = UNK_token
        token2id['SOS'] = SOS_token
        token2id['EOS'] = EOS_token
        id2token[PAD_token] = '<pad>'
        id2token[UNK_token] = '<unk>'
        id2token[SOS_token] = 'SOS'
        id2token[EOS_token] = 'EOS'
        embedding_mat[PAD_token, :] = np.zeros((1,d))
        #generate normal dist 1d array for UNK, SOS, EOS token
        embedding_mat[UNK_token, :] = np.random.normal(size=d)
        embedding_mat[SOS_token, :] = np.random.normal(size=d)
        embedding_mat[EOS_token, :] = np.random.normal(size=d)
        
    return embedding_mat, all_tokens, token2id, id2token

In [11]:
fname_zh = os.getcwd()+'/Embedding/wiki.zh.vec'
fname_eng = os.getcwd()+'/Embedding/wiki-news-300d-1M.vec'
embedding_mat_zh, all_tokens_zh, token2id_zh, id2token_zh = load_embedding(fname_zh, words_to_load)
embedding_mat_en, all_tokens_en, token2id_en, id2token_en = load_embedding(fname_eng, words_to_load)

__Encoder__

In [12]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        embed_mat = torch.from_numpy(embedding_mat_zh).float()
        n, embed_dim = embed_mat.shape
        mask = np.zeros((n,1))
        mask[0] = 1
        mask[1] = 1
        mask[2] = 1
        mask[3] = 1
        mask = torch.from_numpy(mask).float()
        self.mask_embedding = nn.Embedding.from_pretrained(mask, freeze = False)
        self.embedding = nn.Embedding.from_pretrained(embed_mat, freeze = True)
        
        self.gru = nn.GRU(embed_dim, hidden_size, batch_first=True)

    def forward(self, input, input_len, hidden):
        # Compute sorted sequence lengths
        _, idx_sort = torch.sort(input_len, dim=0, descending=True)
        _, idx_unsort = torch.sort(idx_sort, dim=0)
        
        # get embedding of characters
        embed = self.embedding(input)
        mask = self.mask_embedding(input)
        
        embedded = mask*embed + (1-mask)*embed.clone().detach()
        
        # Sort embedding and length
        embedded = embedded.index_select(0, idx_sort)
        input_len = input_len.index_select(0, idx_sort)
        
        packed_emb = nn.utils.rnn.pack_padded_sequence(embedded, input_len.cpu().numpy(), batch_first=True)
        packed_output, hidden = self.gru(packed_emb, hidden)
        output, output_lens =  nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        
        # Unsort output and last hidden unit
        output = output.index_select(0, idx_unsort)
        
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

__Decoder Without Attention__

In [59]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        embed_mat = torch.from_numpy(embedding_mat_en).float()
        n, embed_dim = embed_mat.shape
        mask = np.zeros((n,1))
        mask[0] = 1
        mask[1] = 1
        mask[2] = 1
        mask[3] = 1
        mask = torch.from_numpy(mask).float()
        self.mask_embedding = nn.Embedding.from_pretrained(mask, freeze = False)
        self.embedding = nn.Embedding.from_pretrained(embed_mat, freeze = True)
        
        self.gru = nn.GRU(embed_dim, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, word_input, hidden):
        # get embedding of words
        embed = self.embedding(word_input)
        mask = self.mask_embedding(word_input)
        
        embedded = mask*embed + (1-mask)*embed.clone().detach()
#         print('embed {}'.format(embedded.size()))

        output, hidden = self.gru(embedded, hidden)
#         print('gru input {} hidden {}'.format(output.size(), hidden.size()))
        
        # Final output layer
        output = output.squeeze(1) # B x N
#         print('squeeze {}'.format(output.size()))
        output = self.linear(output)
#         print('linear {}'.format(output.size()))
        output = self.softmax(output)
#         print('softmax {}'.format(output.size()))
#         output = output.max(1)[1]
#         print('max {}'.format(output.size()))

        return output, hidden

__Training__

In [61]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [62]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [63]:
def train(input, target, input_len, target_len, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH, teach_forcing_ratio=0.5):
    encoder_hidden = encoder.initHidden(batch_size)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    max_input_len = max(input_len)
    max_target_len = max(target_len)

    encoder_outputs = torch.zeros([batch_size, max_input_len, encoder.hidden_size], device=device)
    loss = 0

    encoder_output, encoder_hidden = encoder(input, input_len, encoder_hidden)

    decoder_input = torch.tensor([[SOS_token]]*batch_size, device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target[:,di])
            decoder_input = target[:,di].unsqueeze(1)  # Teacher forcing (batch_size, 1)

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach().unsqueeze(1)  # detach from history as input
            loss += criterion(decoder_output, target[:,di])
    #         if decoder_input.item() == EOS_token:
    #             break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / max_target_len

In [64]:
def trainIters(loader, encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        for i, (input, input_len, target, target_len) in enumerate(loader):
            loss = train(input, target, input_len, target_len, encoder, decoder, 
                         encoder_optimizer, decoder_optimizer, criterion, 
                         max_length=MAX_LENGTH, teach_forcing_ratio=teacher_forcing_ratio)
            print_loss_total += loss
            plot_loss_total += loss
            
            
            # TO DO: ADD BLEU
            if i % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print(print_loss_total)
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg))

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

    showPlot(plot_losses)

In [65]:
trainIters(train_loader, encoder, decoder, n_iters=3, print_every=10, plot_every=100, learning_rate=LR_RATE)

tensor(10, device='cuda:0')
0m 0s (- 0m 0s) (1 33%) 1.0000
tensor(100, device='cuda:0')
0m 3s (- 0m 7s) (1 33%) 10.0000
tensor(80, device='cuda:0')
0m 6s (- 0m 12s) (1 33%) 8.0000
tensor(44, device='cuda:0')
0m 9s (- 0m 18s) (1 33%) 4.0000
tensor(29, device='cuda:0')
0m 12s (- 0m 24s) (1 33%) 2.0000
tensor(25, device='cuda:0')
0m 15s (- 0m 30s) (1 33%) 2.0000
tensor(23, device='cuda:0')
0m 18s (- 0m 36s) (1 33%) 2.0000
tensor(24, device='cuda:0')
0m 20s (- 0m 41s) (1 33%) 2.0000
tensor(19, device='cuda:0')
0m 24s (- 0m 48s) (1 33%) 1.0000
tensor(22, device='cuda:0')
0m 26s (- 0m 53s) (1 33%) 2.0000
tensor(20, device='cuda:0')
0m 29s (- 0m 59s) (1 33%) 2.0000
tensor(22, device='cuda:0')
0m 32s (- 1m 4s) (1 33%) 2.0000
tensor(24, device='cuda:0')
0m 34s (- 1m 9s) (1 33%) 2.0000
tensor(21, device='cuda:0')
0m 37s (- 1m 15s) (1 33%) 2.0000
tensor(24, device='cuda:0')
0m 40s (- 1m 20s) (1 33%) 2.0000
tensor(22, device='cuda:0')
0m 43s (- 1m 26s) (1 33%) 2.0000
tensor(20, device='cuda:0')
0m

KeyboardInterrupt: 

In [27]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    """
    Function that generate translation.
    First, feed the source sentence into the encoder and obtain the hidden states from encoder.
    Secondly, feed the hidden states into the decoder and unfold the outputs from the decoder.
    Lastly, for each outputs from the decoder, collect the corresponding words in the target language's vocabulary.
    And collect the attention for each output words.
    @param encoder: the encoder network
    @param decoder: the decoder network
    @param sentence: string, a sentence in source language to be translated
    @param max_length: the max # of words that the decoder can return
    @output decoded_words: a list of words in target language
    @output decoder_attentions: a list of vector, each of which sums up to 1.0
    """    
    # process input sentence
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        # encode the source lanugage
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        # decode the context vector
        decoder_hidden = encoder_hidden # decoder starts from the last encoding sentence
        # output of this function
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            # for each time step, the decoder network takes two inputs: previous outputs and the previous hidden states
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            # hint: print out decoder_output and decoder_attention
            # TODO: add your code here to populate decoded_words and decoder_attentions
            # TODO: do this in 2 ways discussed in class: greedy & beam_search
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            # END TO DO
            
            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [28]:
def evaluateRandomly(encoder, decoder, n=10):
    """
    Randomly select a English sentence from the dataset and try to produce its French translation.
    Note that you need a correct implementation of evaluate() in order to make this function work.
    """    
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [33]:
hidden_size = 256
encoder1 = EncoderRNN(train_input_lang.n_words, hidden_size).to(device)
noattn_decoder1 = DecoderRNN(hidden_size, train_output_lang.n_words).to(device)
# attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
# attn_decoder1 = BahdanauAttnDecoderRNN(hidden_size, output_lang.n_words, n_layers=1, dropout_p=0.1).to(device)

#UNCOMMENT TO TRAIN THE MODEL
# trainIters(encoder1, attn_decoder1, 75000, print_every=5000)
trainIters(encoder1, noattn_decoder1, 75000, print_every=5000)

# encoder1.load_state_dict(torch.load("encoder.pth"))
# attn_decoder1.load_state_dict(torch.load("attn_decoder.pth"))

NameError: name 'tensorsFromPair' is not defined

In [ ]:
# torch.save(encoder1.state_dict(), "encoder.pth")
# torch.save(attn_decoder1.state_dict(), "attn_decoder.pth")

In [ ]:
evaluateRandomly(encoder1, noattn_decoder1)